# Laboratório #4 - Parte II

### Instruções

1. Quando você terminar os exercícios do laboratório, vá ao menu do Jupyter ou Colab e selecione a opção para fazer download do notebook.
    * Os notebooks tem extensão .ipynb. 
    * Este deve ser o arquivo que você irá entregar.
    * No Jupyter vá até a opção **File** -> **Download as** -> **Notebook (.ipynb)**.
    * No Colab vá até a opção **File** -> **Download .ipynb**.
2. Após o download do notebook, vá até a aba de tarefas do MS Teams, localize a tarefa referente a este laboratório e faça o upload do seu notebook. Veja que há uma opção de anexar arquivos à tarefa.

**Nome**:

**Matrícula**:

## Exercícios

### 1) Neste exercício, você irá encontrar uma solução para o problema de regressão linear com a versão estocástica do gradiente descendente.

O código da célula abaixo, além de conter a definição de algumas funções que iremos utilizar neste exercício, gera valores da seguinte **função observável**

$$y_{noisy}(n) = y(n) + w(n),$$

onde $w$ é vetor coluna com $N = 1000$ (ou seja, o número de exemplos) valores retirados de uma distribuição aleatória Gaussiana Normal Padrão (i.e., com média zero e variância unitária) e $y$ é a **função objetivo**. Neste exercício, a **função objetivo** (ou **modelo gerador**) é dada por:

$$y(n) = x_1(n) + x_2(n),$$

onde $x_1$ e $x_2$ são vetores coluna com $N$ valores retirados da distribuição Gaussiana Normal Padrão.

A **função hipótese** para este exercício é dada por

$$h(n) = \hat{a}_1 x_1(n) + \hat{a}_2 x_2(n).$$

De posse destas informações, faça o seguinte:

#### A. Execute a célula de código abaixo.

In [ ]:
# Import all necessary libraries.
import numpy as np
import matplotlib.pyplot as plt
import random

%matplotlib inline

# Reset the pseudo random number generator to the same value.
seed = 42
np.random.seed(seed)
random.seed(seed)

# Generate points for plotting the error surface.
def calculateErrorSurface(X, y_noisy):
    N = len(y_noisy)
    # Generate values for parameters.
    M = 200
    a1 = np.linspace(-20.0, 24.0, M)
    a2 = np.linspace(-20.0, 24.0, M)

    A1, A2 = np.meshgrid(a1, a2)

    # Generate points for plotting the cost-function surface.
    J = np.zeros((M,M))
    for iter1 in range(0, M):
        for iter2 in range(0, M):
            yhat = A1[iter1][iter2]*x1 + A2[iter1][iter2]*x2
            J[iter1][iter2] = (1.0/N)*np.sum(np.square(y_noisy - yhat))
    return J, A1, A2

# Closed-form solution.
def normalEquationSolution(X, y_noisy):
    N = len(y_noisy)
    a_opt = np.linalg.pinv(np.transpose(X).dot(X)).dot(np.transpose(X).dot(y_noisy))
    yhat = X.dot(a_opt)
    Joptimum = (1.0/N)*np.sum(np.power((y_noisy - yhat), 2))
    return a_opt, Joptimum

# Plot performance results.
def plotResults(Jgd, J, A1, A2, a_opt, iteration, maxX, maxY):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10,5))

    cp = ax1.contour(A1, A2, J)
    ax1.clabel(cp, inline=1, fontsize=10)
    ax1.set_xlabel('$a_1$', fontsize=14)
    ax1.set_ylabel('$a_2$', fontsize=14)
    ax1.set_title('Cost-function\'s Contour')
    ax1.plot(a_opt[0], a_opt[1], c='r', marker='*', markersize=14)
    ax1.plot(a_hist[0, 0:iteration], a_hist[1, 0:iteration], 'k--')
    ax1.plot(a_hist[0, 0:iteration], a_hist[1, 0:iteration], 'kx')
    ax1.set_xticks(np.arange(-20, 24, step=4.0))
    ax1.set_yticks(np.arange(-20, 24, step=4.0))

    ax2.plot(np.arange(0, iteration), Jgd[0:iteration])
    ax2.set_yscale('log')
    ax2.set_xlabel('Iteration', fontsize=14)
    ax2.set_ylabel('$J_e$', fontsize=14)
    ax2.set_title('Error vs. Epoch number')
    ax2.set_xlim((0, iteration-1))

    left, bottom, width, height = [0.65, 0.5, 0.23, 0.3]
    ax3 = fig.add_axes([left, bottom, width, height])
    ax3.plot(np.arange(0, maxX), Jgd[0:maxX])
    ax3.set_ylim(0, maxY)

    plt.show()

# Gradient descent solution.
def gradientDescent(X, y_noisy, n_epochs, mb_size, seed):
    # Number of examples.
    N = len(y_noisy)
    
    # Reset PN generator.
    random.seed(seed)
    
    # Random initialization of parameters.
    a = np.array([-20.0, -20.0]).reshape(2, 1)

    # Create vector for parameter history.
    a_hist = np.zeros((2, n_epochs*(N//mb_size)+1))
    # Initialize history vector.
    a_hist[:, 0] = a.reshape(2,)

    # Create array for storing error values.
    Jgd = np.zeros(n_epochs*(N//mb_size)+1)

    Jgd[0] = (1.0/N)*sum(np.power(y_noisy - X.dot(a), 2))

    # Create array for storing gradient values.
    grad_hist = np.zeros((2, n_epochs*(N//mb_size)))

    # Gradient-descent loop.
    inc = 0
    for e in range(n_epochs):

        # Shuffle the whole dataset before every epoch.
        shuffled_indexes = random.sample(range(0, N), N)
        
        for i in range(0, N//mb_size):

            start = i*mb_size
            end = mb_size*(i+1)
            indexes = shuffled_indexes[start:end]

            xi = X[indexes]
            yi = y_noisy[indexes]

            gradients = -(2.0/mb_size)*xi.T.dot(yi - xi.dot(a))
            a = a - alpha*gradients

            Jgd[inc+1] = (1.0/N)*sum(np.power((y_noisy - X.dot(a)), 2))

            grad_hist[:, inc] = gradients.reshape(2,)
            a_hist[:, inc+1] = a.reshape(2,)

            inc += 1
            
    return a, Jgd, a_hist, grad_hist, inc

# Number of examples
N = 1000

# Input values (features)
x1 = np.random.randn(N, 1)
x2 = np.random.randn(N, 1)

# Noise.
w = np.random.randn(N, 1)

# True model.
y = x1 + x2

# Observable function.
y_noisy = y + w

# Concatenate both feature vectors.
X = np.c_[x1,x2]

# Generate values for plotting the error surface.
J, A1, A2 = calculateErrorSurface(X, y_noisy)

# Calculate optimum parameters with closed-form.
a_opt, Joptimum = normalEquationSolution(X, y_noisy)

#### B. Na sequência, faça o seguinte:

   1. Copie o código do item 5 do exercício 1 da parte I do laboratório 4 na célula abaixo.
   2. Altere o parâmetro da função `gradientDescent` que configura o tamanho do mini-batch para que você use a versão **estocástica** do gradiente descendente.
   3. Altere o tamanho do mini-batch (`mb_size`) para $1$.
   4. Altere o passo de aprendizagem para $0.1$.
   5. Altere os 2 últimos parâmetros de entrada da função `plotResults` para $200$ e $10$, respectivamente. Veja abaixo como a função deve ficar:
```python
plotResults(Jgd, J, A1, A2, a_opt, iteration, 200, 10)
```
   6. Em seguida, execute a célula e analise o resultado obtido.

In [1]:
# Cole o código aqui.

#### C. O que você pode dizer sobre o comportamento do algoritmo em relação à versão em batelada do gradiente descendente?

##### Escreva aqui sua análise sobre o comportamento do gradiente descendente estocástico quando comparado com a versão em batelada.

* Resposta:



### 2) Neste exercício, você irá usar a classe `SGDRegressor` da biblioteca SciKit-Learn para resolver um problema de regressão linear. 

O link abaixo contém a versão mais recente da documentação da classe `SGDRegressor`. Antes de resolver o exercício, faça uma leitura rápida da documentação. 

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDRegressor.html

**DICA**: 

* Use o exemplo `SGD_with_scikit_learn_lib.ipynb` como modelo para resolução deste exercício.

Em seguida, faça o seguinte:

#### A. Analise e execute o trecho de código abaixo para gerar o conjunto de treinamento.

In [ ]:
# Número de exemplos.
N = 1000

# Atributo.
x1 = np.random.randn(N, 1)

# Ruído.
w = np.random.randn(N, 1)

# Modelo gerador.
y = 2 + 4*x1

# Função observável.
y_noisy = y + w

#### B. Crie um gráfico comparando os valores originais e ruidosos do modelo gerador. 

In [2]:
# Escreve o código aqui.

#### C. Na célula abaixo, importe a classe `SGDRegressor` e instancia um objeto desta classe.

In [ ]:
# Escreve o código aqui.

#### D. Treine o modelo de regressão linear utilizando o método `fit` do objeto da classe `SGDRegressor`.

In [3]:
# Escreve o código aqui.

#### E. De posse do modelo treinado, use o método `predict` para prever os valores de saída do modelo usando como entrada o atributo `x1`. Armazene o resultado da previsão em uma variável.

In [4]:
# Escreve o código aqui.

#### F. Usando os valores esperados e os valores de saída do modelo de regressão linear, calcule e imprima o erro quadrático médio.

In [5]:
# Escreve o código aqui.

#### G. Imprima os valores dos pesos do modelo obtidos com o treinamento através do gradiente descendente estocástico.

In [6]:
# Escreve o código aqui.

#### H. Crie um gráfico comparando os valores originais, ruidosos e gerados pelo modelo de regressão linear obtido com o gradiente descendente estocástico.

In [7]:
# Escreve o código aqui.